In [1]:
from mylib.statistic_test import *
 
code_id = "0358 - Cell Tracking Table"
loc = join(figpath, code_id)
mkdir(loc)
pass

d:\Software\Anaconda3\envs\maze\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


        D:\Data\FinalResults\0358 - Cell Tracking Table is already existed!


# Number of registered neurons

In [16]:
for i in range(len(f_CellReg_modi)):
    if f_CellReg_modi['Type'][i] != 'Real':
        continue
    
    try:
        with open(f_CellReg_modi['cellreg_folder'][i], 'rb') as handle:
            index_map = pickle.load(handle).astype(np.int64)
    except:
        index_map = ReadCellReg(f_CellReg_modi['cellreg_folder'][i]).astype(np.int64)
    # N: Number of registered cells
    # A: Number of registered cells that were tracked across all sessions
    N = index_map.shape[1]
    A = np.where(np.sum(np.where(index_map > 0, 1, 0), axis=0) == index_map.shape[0])[0].shape[0]
    print(f"{f_CellReg_modi['paradigm'][i]}, Maze {f_CellReg_modi['maze_type'][i]}, Mouse {f_CellReg_modi['MiceID'][i]}:  "
          f" N = {N}, A = {A}, A/N = {A/N:.4}")

CrossMaze, Maze 1, Mouse 10209:   N = 1424, A = 26, A/N = 0.01826
CrossMaze, Maze 1, Mouse 10209:   N = 727, A = 60, A/N = 0.08253
CrossMaze, Maze 2, Mouse 10209:   N = 651, A = 58, A/N = 0.08909
CrossMaze, Maze 1, Mouse 10212:   N = 714, A = 57, A/N = 0.07983
CrossMaze, Maze 1, Mouse 10212:   N = 920, A = 90, A/N = 0.09783
CrossMaze, Maze 2, Mouse 10212:   N = 1589, A = 46, A/N = 0.02895
CrossMaze, Maze 1, Mouse 10224:   N = 2518, A = 36, A/N = 0.0143
CrossMaze, Maze 2, Mouse 10224:   N = 1624, A = 95, A/N = 0.0585
CrossMaze, Maze 1, Mouse 10227:   N = 3154, A = 81, A/N = 0.02568
CrossMaze, Maze 2, Mouse 10227:   N = 1502, A = 154, A/N = 0.1025
CrossMaze, Maze 1, Mouse 10232:   N = 2994, A = 16, A/N = 0.005344
CrossMaze, Maze 2, Mouse 10232:   N = 1945, A = 69, A/N = 0.03548
CrossMaze, Maze 1, Mouse 10234:   N = 7043, A = 0, A/N = 0.0
CrossMaze, Maze 2, Mouse 10234:   N = 3548, A = 0, A/N = 0.0
HairpinMaze, Maze 3, Mouse 10209:   N = 865, A = 40, A/N = 0.04624
HairpinMaze, Maze 3, Mou

C:\Users\Shuyang Yao\AppData\Local\Temp\ipykernel_39976\3354357275.py:7: RuntimeWarning: invalid value encountered in cast
  index_map = pickle.load(handle).astype(np.int64)


# Proportion of registered neurons used for analysis of SFER in the preliminary sense

In [54]:
def registered_neurons_for_historical_analysis(field_reg, field_info, index_map):
    total_n = []
    is_cell = np.where(index_map > 0, 1, 0).astype(np.int64)
    is_used = np.zeros_like(is_cell, np.int64)
    for i in range(field_reg.shape[0]-1):
        for j in range(i+1, field_reg.shape[0]):
            if i == 0:

                base_num = np.where((np.sum(field_reg[i:j, :], axis=0)==j-i)&(field_reg[j, :] == 0))[0]
                active_num = np.where(np.sum(field_reg[i:j+1, :], axis=0)==j-i+1)[0]
                
                base_cells = field_info[:, :, 0][:, base_num].astype(np.int64)
                for m in range(i, j+1):
                    for k in range(base_cells.shape[1]):
                        idx = np.where(index_map[m, :] == base_cells[m, k])[0]
                        is_used[m, idx] = 1
                    
                active_cells = field_info[:, :, 0][:, active_num].astype(np.int64)
                for m in range(i, j+1):
                    for k in range(active_cells.shape[1]):
                        idx = np.where(index_map[m, :] == active_cells[m, k])[0]
                        is_used[m, idx] = 1
                    
            elif i == 1:
                base_num = np.where((np.sum(field_reg[i:j, :], axis=0)==j-i)&(field_reg[j, :] == 0)&(field_reg[i-1, :] != 1))[0]
                active_num = np.where((np.sum(field_reg[i:j+1, :], axis=0)==j-i+1)&(field_reg[i-1, :] != 1))[0]
                
                base_cells = field_info[:, :, 0][:, base_num].astype(np.int64)
                for m in range(i, j+1):
                    for k in range(base_cells.shape[1]):
                        idx = np.where(index_map[m, :] == base_cells[m, k])[0]
                        is_used[m, idx] = 1
                    
                active_cells = field_info[:, :, 0][:, active_num].astype(np.int64)
                for m in range(i, j+1):
                    for k in range(active_cells.shape[1]):
                        idx = np.where(index_map[m, :] == active_cells[m, k])[0]
                        is_used[m, idx] = 1
                
            else:
                base_num = np.where((np.sum(field_reg[i:j, :], axis=0)==j-i)&(field_reg[j, :] == 0)&
                                        ((field_reg[i-1, :] == 0)|((np.isnan(field_reg[i-1, :]))&
                                                                     (field_reg[i-2, :] != 1))))[0]
                active_num = np.where((np.sum(field_reg[i:j+1, :], axis=0)==j-i+1)&
                                          ((field_reg[i-1, :] == 0)|((np.isnan(field_reg[i-1, :]))&
                                                                     (field_reg[i-2, :] != 1))))[0]
                base_cells = field_info[:, :, 0][:, base_num].astype(np.int64)
                for m in range(i, j+1):
                    for k in range(base_cells.shape[1]):
                        idx = np.where(index_map[m, :] == base_cells[m, k])[0]
                        is_used[m, idx] = 1
                    
                active_cells = field_info[:, :, 0][:, active_num].astype(np.int64)
                for m in range(i, j+1):
                    for k in range(active_cells.shape[1]):
                        idx = np.where(index_map[m, :] == active_cells[m, k])[0]
                        is_used[m, idx] = 1
    
    is_used[np.where(is_cell == 0)] = 0
    return np.sum(is_used), np.sum(is_cell)

avg_percent = []
for i in range(len(f_CellReg_modi)):
    if f_CellReg_modi['Type'][i] != 'Real':
        continue
    
    try:
        with open(f_CellReg_modi['cellreg_folder'][i], 'rb') as handle:
            index_map = pickle.load(handle).astype(np.int64)
    except:
        index_map = ReadCellReg(f_CellReg_modi['cellreg_folder'][i]).astype(np.int64)
    
    with open(f_CellReg_modi['Trace File'][i], 'rb') as handle:
        trace_mdays = pickle.load(handle)
    
    try:
        field_reg = trace_mdays['field_reg']
        field_info = trace_mdays['field_info']

        A, N = registered_neurons_for_historical_analysis(field_reg, field_info, index_map)
        print(f"{f_CellReg_modi['paradigm'][i]}, Maze {f_CellReg_modi['maze_type'][i]}, Mouse {f_CellReg_modi['MiceID'][i]}:  "
              f"  N = {A}/{N} ({A/N*100:.3}%)")
        avg_percent.append(A/N*100)
    except:
        field_reg = trace_mdays['cis']['field_reg']
        field_info = trace_mdays['cis']['field_info']
        A, N = registered_neurons_for_historical_analysis(field_reg, field_info, index_map)
        print(f"{f_CellReg_modi['paradigm'][i]}, Maze {f_CellReg_modi['maze_type'][i]}, Mouse {f_CellReg_modi['MiceID'][i]}:  "
              f"  cis: N = {A}/{N} ({A/N*100:.3}%)")
        avg_percent.append(A/N*100)
        field_reg = trace_mdays['trs']['field_reg']
        field_info = trace_mdays['trs']['field_info']
        A, N = registered_neurons_for_historical_analysis(field_reg, field_info, index_map)
        print(f"{f_CellReg_modi['paradigm'][i]}, Maze {f_CellReg_modi['maze_type'][i]}, Mouse {f_CellReg_modi['MiceID'][i]}:  "
              f"  trs: N = {A}/{N} ({A/N*100:.3}%)")
        avg_percent.append(A/N*100)
    
print_estimator(np.array(avg_percent))

CrossMaze, Maze 1, Mouse 10209:    N = 2017/3629 (55.6%)
CrossMaze, Maze 1, Mouse 10209:    N = 2000/2923 (68.4%)
CrossMaze, Maze 2, Mouse 10209:    N = 2403/3361 (71.5%)
CrossMaze, Maze 1, Mouse 10212:    N = 2384/3085 (77.3%)
CrossMaze, Maze 1, Mouse 10212:    N = 3787/4664 (81.2%)
CrossMaze, Maze 2, Mouse 10212:    N = 2960/4457 (66.4%)
CrossMaze, Maze 1, Mouse 10224:    N = 7422/10895 (68.1%)
CrossMaze, Maze 2, Mouse 10224:    N = 3844/5466 (70.3%)


C:\Users\Shuyang Yao\AppData\Local\Temp\ipykernel_39976\3507566039.py:69: RuntimeWarning: invalid value encountered in cast
  index_map = pickle.load(handle).astype(np.int64)


CrossMaze, Maze 1, Mouse 10227:    N = 13054/16822 (77.6%)
CrossMaze, Maze 2, Mouse 10227:    N = 7012/8533 (82.2%)
CrossMaze, Maze 1, Mouse 10232:    N = 10751/14486 (74.2%)
CrossMaze, Maze 2, Mouse 10232:    N = 4874/6733 (72.4%)
CrossMaze, Maze 1, Mouse 10234:    N = 4327/11679 (37.0%)
CrossMaze, Maze 2, Mouse 10234:    N = 1984/5590 (35.5%)
HairpinMaze, Maze 3, Mouse 10209:    cis: N = 777/2008 (38.7%)
HairpinMaze, Maze 3, Mouse 10209:    trs: N = 693/2008 (34.5%)
HairpinMaze, Maze 3, Mouse 10212:    cis: N = 1381/2354 (58.7%)
HairpinMaze, Maze 3, Mouse 10212:    trs: N = 1321/2354 (56.1%)
HairpinMaze, Maze 3, Mouse 10224:    cis: N = 2299/3022 (76.1%)
HairpinMaze, Maze 3, Mouse 10224:    trs: N = 2326/3022 (77.0%)
HairpinMaze, Maze 3, Mouse 10227:    cis: N = 3212/4667 (68.8%)
HairpinMaze, Maze 3, Mouse 10227:    trs: N = 3348/4667 (71.7%)
ReverseMaze, Maze 1, Mouse 10209:    cis: N = 1701/3081 (55.2%)
ReverseMaze, Maze 1, Mouse 10209:    trs: N = 1616/3081 (52.5%)
ReverseMaze, Ma

# Number of registered fields
and the proportion of registered fields used for the analysis of SFER in the preliminary sense

In [2]:
def registered_fields_for_historical_analysis(field_reg):
    total_n = []
    for i in range(field_reg.shape[0]-1):
        for j in range(i+1, field_reg.shape[0]):
            if i == 0:

                base_num = np.where((np.sum(field_reg[i:j, :], axis=0)==j-i)&(field_reg[j, :] == 0))[0]
                active_num = np.where(np.sum(field_reg[i:j+1, :], axis=0)==j-i+1)[0]
                total_n.append(active_num)
                total_n.append(base_num)
                    
            elif i == 1:
                base_num = np.where((np.sum(field_reg[i:j, :], axis=0)==j-i)&(field_reg[j, :] == 0)&(field_reg[i-1, :] != 1))[0]
                active_num = np.where((np.sum(field_reg[i:j+1, :], axis=0)==j-i+1)&(field_reg[i-1, :] != 1))[0]
                total_n.append(active_num)
                total_n.append(base_num)
                
            else:
                base_num = np.where((np.sum(field_reg[i:j, :], axis=0)==j-i)&(field_reg[j, :] == 0)&
                                        ((field_reg[i-1, :] == 0)|((np.isnan(field_reg[i-1, :]))&
                                                                     (field_reg[i-2, :] != 1))))[0]
                active_num = np.where((np.sum(field_reg[i:j+1, :], axis=0)==j-i+1)&
                                          ((field_reg[i-1, :] == 0)|((np.isnan(field_reg[i-1, :]))&
                                                                     (field_reg[i-2, :] != 1))))[0]
                total_n.append(active_num)
                total_n.append(base_num)
    return np.unique(np.concatenate(total_n)).shape[0]

avg_percent = []
for i in range(len(f_CellReg_modi)):
    if f_CellReg_modi['Type'][i] != 'Real':
        continue
    
    with open(f_CellReg_modi['Trace File'][i], 'rb') as handle:
        trace_mdays = pickle.load(handle)
    
    output = f"{f_CellReg_modi['paradigm'][i]}, Maze {f_CellReg_modi['maze_type'][i]}, Mouse {f_CellReg_modi['MiceID'][i]}:  "
    try:
        A = registered_fields_for_historical_analysis(trace_mdays['field_reg'])
        B = trace_mdays['field_reg'].shape[1]
        avg_percent.append(A/B*100)
        print(f"{output}\n  {B} {A}/{A/B:.4}")
    except:
        A = registered_fields_for_historical_analysis(trace_mdays['cis']['field_reg'])
        B = trace_mdays['cis']['field_reg'].shape[1]
        avg_percent.append(A/B*100)
        C = registered_fields_for_historical_analysis(trace_mdays['trs']['field_reg'])
        D = trace_mdays['trs']['field_reg'].shape[1]
        avg_percent.append(C/D*100)
        print(f"{output}\n  cis: {B} {A}/{A/B:.4}")
        print(f"  trs: {D} {C}/{C/D:.4}")

print_estimator(np.array(avg_percent))
    


CrossMaze, Maze 1, Mouse 10209:  
  13018 4105/0.3153
CrossMaze, Maze 1, Mouse 10209:  
  6269 3190/0.5089
CrossMaze, Maze 2, Mouse 10209:  
  6874 4062/0.5909
CrossMaze, Maze 1, Mouse 10212:  
  11282 5906/0.5235
CrossMaze, Maze 1, Mouse 10212:  
  12064 6949/0.576
CrossMaze, Maze 2, Mouse 10212:  
  16178 6241/0.3858
CrossMaze, Maze 1, Mouse 10224:  
  29562 10810/0.3657
CrossMaze, Maze 2, Mouse 10224:  
  15902 6888/0.4332
CrossMaze, Maze 1, Mouse 10227:  
  38086 17535/0.4604
CrossMaze, Maze 2, Mouse 10227:  
  18797 11855/0.6307
CrossMaze, Maze 1, Mouse 10232:  
  21614 10888/0.5037
CrossMaze, Maze 2, Mouse 10232:  
  14478 7019/0.4848
CrossMaze, Maze 1, Mouse 10234:  
  43481 6362/0.1463
CrossMaze, Maze 2, Mouse 10234:  
  18960 3297/0.1739
HairpinMaze, Maze 3, Mouse 10209:  
  cis: 4400 1707/0.388
  trs: 3539 1430/0.4041
HairpinMaze, Maze 3, Mouse 10212:  
  cis: 8054 3942/0.4894
  trs: 7323 3463/0.4729
HairpinMaze, Maze 3, Mouse 10224:  
  cis: 6227 4294/0.6896
  trs: 5812 4004

# The number of registered fields used for SFER in the general sense

In [2]:
def registered_fields_for_general_analysis(field_reg, is_gate = True):
        sequences = []
        indices = []
        for i in range(field_reg.shape[0]-1):
            for j in range(i+1, field_reg.shape[0]):
                if i == 0 and j != field_reg.shape[0]-1:
                    idx = np.where(
                        (np.isnan(np.sum(field_reg[i:j+1, :], axis=0)) == False) &
                        (np.isnan(field_reg[j+1, :]) == True)
                    )[0]
                    for k in idx:
                        sequences.append(field_reg[i:j+1, k].astype(np.int64))
                        indices.append(k)
                    
                elif i != 0 and j == field_reg.shape[0]-1:
                    idx = np.where(
                        (np.isnan(np.sum(field_reg[i:j+1, :], axis=0)) == False) &
                        (np.isnan(field_reg[i-1, :]) == True)
                    )[0]
                    for k in idx:
                        sequences.append(field_reg[i:j+1, k].astype(np.int64))
                        indices.append(k)
                
                elif i == 0 and j == field_reg.shape[0]-1:
                    idx = np.where(
                        (np.isnan(np.sum(field_reg[i:j+1, :], axis=0)) == False)
                    )[0]
                    for k in idx:
                        sequences.append(field_reg[i:j+1, k].astype(np.int64))
                        indices.append(k)
                else:
                    idx = np.where(
                        (np.isnan(np.sum(field_reg[i:j+1, :], axis=0)) == False) &
                        (np.isnan(field_reg[i-1, :]) == True) &
                        (np.isnan(field_reg[j+1, :]) == True)
                    )[0]
                    for k in idx:
                        sequences.append(field_reg[i:j+1, k].astype(np.int64))
                        indices.append(k)
                    
        # Cut the sequence if 0 continuously occurs for 8 times
        if is_gate:
            for i in range(len(sequences)-1):
                num_zero = 0
                is_split = False
                split_point = []
                for j in range(sequences[i].shape[0]):
                    if num_zero == 8:
                        is_split = True
                
                    if sequences[i][j] == 0:
                        num_zero += 1
                    else:
                        if is_split:
                            split_point.append(j)
                            is_split = False
                        num_zero = 0
            
                if len(split_point) != 0:      
                    split_point = [0] + split_point + [sequences[i].shape[0]]
                    for j in range(1, len(split_point)-1):
                        sequences.append(sequences[i][split_point[j]:split_point[j+1]])
                        indices.append(indices[i])
                        
                    sequences[i] = sequences[i][split_point[0]:split_point[1]]
        
        # Ensure every sequence has a length of at least 2, starting from 1.
        for i in range(len(sequences)-1, -1, -1):
            if np.sum(sequences[i]) == 0:
                sequences.pop(i)
                indices.pop(i)
                continue
    
            for j in range(sequences[i].shape[0]):
                if sequences[i][j] == 1:
                    sequences[i] = sequences[i][j:]

                    if sequences[i].shape[0] <= 1:
                        sequences.pop(i)
                        indices.pop(i)
                    break
        
        sequences_len = np.array([seq.shape[0] for seq in sequences], dtype=np.int64)
        indices = np.array(indices, dtype=np.int64)
        if max(sequences_len) > 12:
            return np.where(sequences_len >= 10)[0].shape[0]
        else:
            return np.where(sequences_len >= 5)[0].shape[0]

avg_percent = []
for i in range(len(f_CellReg_modi)):
    if f_CellReg_modi['Type'][i] != 'Real':
        continue
    
    with open(f_CellReg_modi['Trace File'][i], 'rb') as handle:
        trace_mdays = pickle.load(handle)
    
    output = f"{f_CellReg_modi['paradigm'][i]}, Maze {f_CellReg_modi['maze_type'][i]}, Mouse {f_CellReg_modi['MiceID'][i]}:  "
    try:
        A = registered_fields_for_general_analysis(trace_mdays['field_reg'], is_gate=True)
        B = trace_mdays['field_reg'].shape[1]
        avg_percent.append(A/B*100)
        print(f"{output}\n  N: {trace_mdays['field_reg'].shape[1]}  Used: {A}/{A/B*100:.3}%")
    except:
        C = registered_fields_for_general_analysis(trace_mdays['cis']['field_reg'], is_gate=True)
        D = trace_mdays['cis']['field_reg'].shape[1]
        avg_percent.append(C/D*100)
        print(f"{output}\n  cis: {D} {C}/{C/D*100:.3}%")
        E = registered_fields_for_general_analysis(trace_mdays['trs']['field_reg'], is_gate=True)
        F = trace_mdays['trs']['field_reg'].shape[1]
        avg_percent.append(E/F*100)
        print(f"  trs: {F} {E}/{E/F*100:.3}%")
        
print_estimator(np.array(avg_percent))

CrossMaze, Maze 1, Mouse 10209:  
  N: 13018  Used: 717/5.51%
CrossMaze, Maze 1, Mouse 10209:  
  N: 6269  Used: 788/12.6%
CrossMaze, Maze 2, Mouse 10209:  
  N: 6874  Used: 838/12.2%
CrossMaze, Maze 1, Mouse 10212:  
  N: 11282  Used: 3643/32.3%
CrossMaze, Maze 1, Mouse 10212:  
  N: 12064  Used: 1556/12.9%
CrossMaze, Maze 2, Mouse 10212:  
  N: 16178  Used: 1062/6.56%
CrossMaze, Maze 1, Mouse 10224:  
  N: 29562  Used: 3736/12.6%
CrossMaze, Maze 2, Mouse 10224:  
  N: 15902  Used: 1762/11.1%
CrossMaze, Maze 1, Mouse 10227:  
  N: 38086  Used: 6416/16.8%
CrossMaze, Maze 2, Mouse 10227:  
  N: 18797  Used: 2233/11.9%
CrossMaze, Maze 1, Mouse 10232:  
  N: 21614  Used: 2768/12.8%
CrossMaze, Maze 2, Mouse 10232:  
  N: 14478  Used: 891/6.15%
CrossMaze, Maze 1, Mouse 10234:  
  N: 43481  Used: 890/2.05%
CrossMaze, Maze 2, Mouse 10234:  
  N: 18960  Used: 941/4.96%
HairpinMaze, Maze 3, Mouse 10209:  
  cis: 4400 549/12.5%
  trs: 3539 467/13.2%
HairpinMaze, Maze 3, Mouse 10212:  
  cis: 805

# Percentage of fields included in the analysis

In [53]:
def fields_for_historical_analysis(field_reg):
    is_field = np.where(field_reg > 0, 1, 0).astype(np.int64)
    is_used = np.zeros_like(is_field, np.int64)
    for i in range(field_reg.shape[0]-1):
        for j in range(i+1, field_reg.shape[0]):
            if i == 0:

                base_num = np.where((np.sum(field_reg[i:j, :], axis=0)==j-i)&(field_reg[j, :] == 0))[0]
                active_num = np.where(np.sum(field_reg[i:j+1, :], axis=0)==j-i+1)[0]
                is_used[i:j, base_num] = 1
                is_used[i:j+1, active_num] = 1
                    
            elif i == 1:
                base_num = np.where((np.sum(field_reg[i:j, :], axis=0)==j-i)&(field_reg[j, :] == 0)&(field_reg[i-1, :] != 1))[0]
                active_num = np.where((np.sum(field_reg[i:j+1, :], axis=0)==j-i+1)&(field_reg[i-1, :] != 1))[0]
                is_used[i:j, base_num] = 1
                is_used[i:j+1, active_num] = 1
                
            else:
                base_num = np.where((np.sum(field_reg[i:j, :], axis=0)==j-i)&(field_reg[j, :] == 0)&
                                        ((field_reg[i-1, :] == 0)|((np.isnan(field_reg[i-1, :]))&
                                                                     (field_reg[i-2, :] != 1))))[0]
                active_num = np.where((np.sum(field_reg[i:j+1, :], axis=0)==j-i+1)&
                                          ((field_reg[i-1, :] == 0)|((np.isnan(field_reg[i-1, :]))&
                                                                     (field_reg[i-2, :] != 1))))[0]
                is_used[i:j, base_num] = 1
                is_used[i:j+1, active_num] = 1
    
    is_used[np.where(is_field == 0)] = 0
    return np.sum(is_used), np.sum(is_field)

avg_percent = []
for i in range(len(f_CellReg_modi)):
    if f_CellReg_modi['Type'][i] != 'Real':
        continue
    
    with open(f_CellReg_modi['Trace File'][i], 'rb') as handle:
        trace_mdays = pickle.load(handle)
    
    output = f"{f_CellReg_modi['paradigm'][i]}, Maze {f_CellReg_modi['maze_type'][i]}, Mouse {f_CellReg_modi['MiceID'][i]}:  "
    try:
        A, B = fields_for_historical_analysis(trace_mdays['field_reg'])
        avg_percent.append(A/B*100)
        
        print(f"{output}\n  {A}/{B} {A/B*100:.3}%")
    except:
        A, B = fields_for_historical_analysis(trace_mdays['cis']['field_reg'])
        C, D = fields_for_historical_analysis(trace_mdays['trs']['field_reg'])
        print(f"{output}\n  cis: {A}/{B} {A/B*100:.3}%")
        print(f"  trs: {C}/{D} {C/D*100:.3}%")
        avg_percent.append(A/B*100)
        avg_percent.append(C/D*100)
    
print_estimator(np.array(avg_percent))

CrossMaze, Maze 1, Mouse 10209:  
  13686/27732 49.4%
CrossMaze, Maze 1, Mouse 10209:  
  10681/16446 64.9%
CrossMaze, Maze 2, Mouse 10209:  
  14282/21073 67.8%
CrossMaze, Maze 1, Mouse 10212:  
  23386/37340 62.6%
CrossMaze, Maze 1, Mouse 10212:  
  29795/42020 70.9%
CrossMaze, Maze 2, Mouse 10212:  
  22562/39378 57.3%
CrossMaze, Maze 1, Mouse 10224:  
  51142/89195 57.3%
CrossMaze, Maze 2, Mouse 10224:  
  24021/39255 61.2%
CrossMaze, Maze 1, Mouse 10227:  
  78259/120792 64.8%
CrossMaze, Maze 2, Mouse 10227:  
  35653/51260 69.6%
CrossMaze, Maze 1, Mouse 10232:  
  38155/60492 63.1%
CrossMaze, Maze 2, Mouse 10232:  
  18405/30168 61.0%
CrossMaze, Maze 1, Mouse 10234:  
  19583/64825 30.2%
CrossMaze, Maze 2, Mouse 10234:  
  7453/25826 28.9%
HairpinMaze, Maze 3, Mouse 10209:  
  cis: 3408/6883 49.5%
  trs: 2548/5078 50.2%
HairpinMaze, Maze 3, Mouse 10212:  
  cis: 8802/14404 61.1%
  trs: 7202/12348 58.3%
HairpinMaze, Maze 3, Mouse 10224:  
  cis: 8515/12069 70.6%
  trs: 8062/11354 

In [59]:
def fields_for_general_analysis(field_reg, is_gate = True):
        sequences = []
        is_fields = np.where(field_reg > 0, 1, 0).astype(np.int64)
        is_used = np.zeros_like(is_fields, np.int64)
        indices = []
        for i in range(field_reg.shape[0]-1):
            for j in range(i+1, field_reg.shape[0]):
                if i == 0 and j != field_reg.shape[0]-1:
                    idx = np.where(
                        (np.isnan(np.sum(field_reg[i:j+1, :], axis=0)) == False) &
                        (np.isnan(field_reg[j+1, :]) == True)
                    )[0]
                    for k in idx:
                        sequences.append(field_reg[i:j+1, k].astype(np.int64))
                        indices.append((i, j+1, k))
                    
                elif i != 0 and j == field_reg.shape[0]-1:
                    idx = np.where(
                        (np.isnan(np.sum(field_reg[i:j+1, :], axis=0)) == False) &
                        (np.isnan(field_reg[i-1, :]) == True)
                    )[0]
                    for k in idx:
                        sequences.append(field_reg[i:j+1, k].astype(np.int64))
                        indices.append((i, j+1, k))
                
                elif i == 0 and j == field_reg.shape[0]-1:
                    idx = np.where(
                        (np.isnan(np.sum(field_reg[i:j+1, :], axis=0)) == False)
                    )[0]
                    for k in idx:
                        sequences.append(field_reg[i:j+1, k].astype(np.int64))
                        indices.append((i, j+1, k))
                else:
                    idx = np.where(
                        (np.isnan(np.sum(field_reg[i:j+1, :], axis=0)) == False) &
                        (np.isnan(field_reg[i-1, :]) == True) &
                        (np.isnan(field_reg[j+1, :]) == True)
                    )[0]
                    for k in idx:
                        sequences.append(field_reg[i:j+1, k].astype(np.int64))
                        indices.append((i, j+1, k))
                    
        # Cut the sequence if 0 continuously occurs for 8 times
        if is_gate:
            for i in range(len(sequences)-1):
                num_zero = 0
                is_split = False
                split_point = []
                for j in range(sequences[i].shape[0]):
                    if num_zero == 8:
                        is_split = True
                
                    if sequences[i][j] == 0:
                        num_zero += 1
                    else:
                        if is_split:
                            split_point.append(j)
                            is_split = False
                        num_zero = 0
            
                if len(split_point) != 0:      
                    split_point = [0] + split_point + [sequences[i].shape[0]]
                    for j in range(1, len(split_point)-1):
                        sequences.append(sequences[i][split_point[j]:split_point[j+1]])
                        l, m, n = indices[i]
                        indices.append((l + split_point[j], l + split_point[j+1], n))
                        
                    sequences[i] = sequences[i][split_point[0]:split_point[1]]
                    indices[i] = (indices[i][0] + split_point[0], indices[i][0] + split_point[1], indices[i][2])
        
        # Ensure every sequence has a length of at least 2, starting from 1.
        for i in range(len(sequences)-1, -1, -1):
            if np.sum(sequences[i]) == 0:
                sequences.pop(i)
                indices.pop(i)
                continue
    
            for j in range(sequences[i].shape[0]):
                if sequences[i][j] == 1:
                    sequences[i] = sequences[i][j:]

                    if sequences[i].shape[0] <= 1:
                        sequences.pop(i)
                        indices.pop(i)
                    break
        
        sequences_len = np.array([seq.shape[0] for seq in sequences], dtype=np.int64)
        indices = np.array(indices, dtype=np.int64)
        if max(sequences_len) > 12:
            idx = np.where(sequences_len >= 10)[0]
        else:
            idx = np.where(sequences_len >= 5)[0]
            
        for i in idx:
            l, m, n = indices[i]
            is_used[l:m, n] = 1
        
        is_used[np.where(is_fields == 0)] = 0
            
        return np.sum(is_used), np.sum(is_fields)

avg_percent = []
for i in range(len(f_CellReg_modi)):
    if f_CellReg_modi['Type'][i] != 'Real':
        continue
    
    with open(f_CellReg_modi['Trace File'][i], 'rb') as handle:
        trace_mdays = pickle.load(handle)
    
    output = f"{f_CellReg_modi['paradigm'][i]}, Maze {f_CellReg_modi['maze_type'][i]}, Mouse {f_CellReg_modi['MiceID'][i]}:  "
    try:
        A, B = fields_for_general_analysis(trace_mdays['field_reg'], is_gate=True)
        print(f"{output}\n  {A}/{B} ({A/B*100:.3}%)")
        avg_percent.append(A/B*100)
    except:
        
        A, B = fields_for_general_analysis(trace_mdays['cis']['field_reg'], is_gate=True)
        C, D = fields_for_general_analysis(trace_mdays['trs']['field_reg'], is_gate=True)
        avg_percent.append(A/B*100)
        avg_percent.append(C/D*100)
        print(f"{output}\n  cis: {A}/{B} ({A/B*100:.3}%)")
        print(f"  trs: {C}/{D} ({C/D*100:.3}%)")
        
print_estimator(np.array(avg_percent))

CrossMaze, Maze 1, Mouse 10209:  
  4741/27732 (17.1%)
CrossMaze, Maze 1, Mouse 10209:  
  5032/16446 (30.6%)
CrossMaze, Maze 2, Mouse 10209:  
  5556/21073 (26.4%)
CrossMaze, Maze 1, Mouse 10212:  
  19702/37340 (52.8%)
CrossMaze, Maze 1, Mouse 10212:  
  12386/42020 (29.5%)
CrossMaze, Maze 2, Mouse 10212:  
  7722/39378 (19.6%)
CrossMaze, Maze 1, Mouse 10224:  
  28969/89195 (32.5%)
CrossMaze, Maze 2, Mouse 10224:  
  10372/39255 (26.4%)
CrossMaze, Maze 1, Mouse 10227:  
  42689/120792 (35.3%)
CrossMaze, Maze 2, Mouse 10227:  
  10917/51260 (21.3%)
CrossMaze, Maze 1, Mouse 10232:  
  16438/60492 (27.2%)
CrossMaze, Maze 2, Mouse 10232:  
  4013/30168 (13.3%)
CrossMaze, Maze 1, Mouse 10234:  
  5783/64825 (8.92%)
CrossMaze, Maze 2, Mouse 10234:  
  3411/25826 (13.2%)
HairpinMaze, Maze 3, Mouse 10209:  
  cis: 1561/6883 (22.7%)
  trs: 1053/5078 (20.7%)
HairpinMaze, Maze 3, Mouse 10212:  
  cis: 4227/14404 (29.3%)
  trs: 3231/12348 (26.2%)
HairpinMaze, Maze 3, Mouse 10224:  
  cis: 3823/